# Expliquer les Préférences : Problème à 27 critères

**Objectif** : Expliquer pourquoi la **Solution 1** est meilleure que les Solutions 2, 3 et 4.

Ce notebook utilise la méthode des **trade-offs** pour décomposer chaque comparaison en arguments simples et compréhensibles.

## Contexte

Nous avons un problème de décision multicritère avec :
- **27 critères** nommés a, b, c, ..., y, z, A
- **4 solutions** candidates
- Des **poids** associés à chaque critère

Le score de chaque solution est calculé par somme pondérée.

In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from typing import List, Set, Optional, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

## 1. Chargement des données

Le fichier Excel contient :
- Ligne 5 (index 4) : noms des critères
- Ligne 6 (index 5) : poids des critères
- Lignes 8-11 (index 7-10) : valeurs des 4 solutions

In [2]:
# Charger les données depuis Excel
df_raw = pd.read_excel('content/data27crit.xlsx', header=None)

# Extraire les noms des critères (ligne index 4, colonnes 1-27)
criteria_names = df_raw.iloc[4, 1:28].tolist()

# Extraire les poids (ligne index 5, colonnes 1-27)
weights = df_raw.iloc[5, 1:28].values.astype(float)

# Extraire les solutions (lignes index 7-10, colonnes 1-27)
solution_names = ['Solution 1', 'Solution 2', 'Solution 3', 'Solution 4']
solutions = {}
for i, name in enumerate(solution_names):
    solutions[name] = df_raw.iloc[7 + i, 1:28].values.astype(float)

# Créer la matrice X
X = np.array([solutions[name] for name in solution_names])

print(f"Dataset : {len(solution_names)} solutions, {len(criteria_names)} critères")
print(f"\nCritères : {', '.join(criteria_names)}")
print(f"\nPoids : ")
for name, w in zip(criteria_names, weights):
    print(f"  {name}: {w}")

Dataset : 4 solutions, 27 critères

Critères : a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z, A

Poids : 
  a: 2.0
  b: 1.0
  c: 5.0
  d: 3.0
  e: 4.0
  f: 6.0
  g: 1.0
  h: 7.0
  i: 4.0
  j: 3.0
  k: 8.0
  l: 5.0
  m: 1.0
  n: 2.0
  o: 1.0
  p: 4.0
  q: 5.0
  r: 6.0
  s: 2.0
  t: 7.0
  u: 1.0
  v: 2.0
  w: 6.0
  x: 3.0
  y: 5.0
  z: 5.0
  A: 1.0


In [3]:
# Calculer les scores de chaque solution
scores = {name: np.dot(weights, solutions[name]) for name in solution_names}

print("Scores des solutions (somme pondérée) :")
print("=" * 50)
for name in sorted(scores, key=lambda x: scores[x], reverse=True):
    print(f"  {name}: {scores[name]:.2f}")

print(f"\n→ La Solution 1 est-elle la meilleure ? {scores['Solution 1'] == max(scores.values())}")

Scores des solutions (somme pondérée) :
  Solution 1: 19725.30
  Solution 2: 17653.30
  Solution 4: 15711.30
  Solution 3: -15731.30

→ La Solution 1 est-elle la meilleure ? True


In [4]:
# Afficher le tableau complet des solutions
df_solutions = pd.DataFrame(solutions, index=criteria_names).T
df_solutions['Score'] = [scores[name] for name in solution_names]
print("Tableau des solutions :")
print(df_solutions)

Tableau des solutions :
                a      b       c           d      e           f      g  \
Solution 1  532.0  120.0   330.0   54.000000   12.0  125.000000  320.0   
Solution 2   94.0 -756.0   174.4 -254.666667  225.5  162.333333  670.0   
Solution 3  174.0  652.0 -1094.0 -176.000000 -854.0 -573.000000    6.0   
Solution 4  604.0  976.0   680.0  -52.000000 -234.0  333.000000 -254.0   

                     h      i           j  ...      s      t      u      v  \
Solution 1   59.400000  125.0  325.000000  ...  577.0  231.0  677.0   32.0   
Solution 2   19.971429  127.5   37.666667  ...  155.0  349.0  363.0  206.0   
Solution 3   50.600000  515.0 -395.000000  ...  107.0  -37.0 -483.0  706.0   
Solution 4 -608.600000 -523.0  277.000000  ... -295.0  769.0  -39.0 -192.0   

                w      x      y      z      A    Score  
Solution 1  197.0  123.0  224.0  324.1   53.0  19725.3  
Solution 2  294.0  201.0  214.8  142.9  465.0  17653.3  
Solution 3 -301.0   27.0  490.0 -922.1 -717

## 2. Fonctions utilitaires

### Définitions

Pour comparer deux solutions A et B :
- **omega[i]** = weight[i] × (A[i] - B[i])
- **pros** = critères où A est meilleur (omega > 0)
- **cons** = critères où A est moins bon (omega < 0)
- **neutral** = critères égaux

In [5]:
EPS = 1e-6

def compute_omega(A: np.ndarray, B: np.ndarray, weights: np.ndarray) -> np.ndarray:
    """
    Calcule les contributions de chaque critère.
    omega_i = w_i * (A_i - B_i)
    sum(omega) = score(A) - score(B) > 0  <=>  A est meilleur que B
    """
    return weights * (A - B)


def classify_omega(omega: np.ndarray, eps: float = EPS) -> Tuple[List[int], List[int], List[int]]:
    """Classifie les critères en pros, cons, neutral."""
    pros = [i for i, v in enumerate(omega) if v > eps]
    cons = [i for i, v in enumerate(omega) if v < -eps]
    neutral = [i for i, v in enumerate(omega) if abs(v) <= eps]
    return pros, cons, neutral


def display_comparison(name_A: str, name_B: str, solutions: dict, 
                       weights: np.ndarray, criteria_names: List[str]):
    """Affiche les détails d'une comparaison."""
    A = solutions[name_A]
    B = solutions[name_B]
    
    omega = compute_omega(A, B, weights)
    pros, cons, neutral = classify_omega(omega)
    
    score_A = np.dot(weights, A)
    score_B = np.dot(weights, B)
    
    print(f"\n{'='*80}")
    print(f"COMPARAISON : {name_A} vs {name_B}")
    print(f"{'='*80}")
    
    print(f"\nScores : {name_A} = {score_A:.2f}, {name_B} = {score_B:.2f}")
    print(f"Différence : {score_A - score_B:.2f}")
    
    print(f"\n{'Critère':<8} {'Poids':>8} {name_A:>12} {name_B:>12} {'Omega':>12} {'Type':<12}")
    print("-" * 70)
    
    for i, crit in enumerate(criteria_names):
        if omega[i] > EPS:
            type_str = "PRO"
        elif omega[i] < -EPS:
            type_str = "CON"
        else:
            type_str = "NEUTRAL"
        print(f"{crit:<8} {weights[i]:>8.1f} {A[i]:>12.2f} {B[i]:>12.2f} {omega[i]:>+12.2f} {type_str:<12}")
    
    print(f"\nRésumé :")
    print(f"  - Pros (avantages) : {len(pros)} critères")
    print(f"  - Cons (inconvénients) : {len(cons)} critères")
    print(f"  - Neutres : {len(neutral)} critères")
    
    return omega, pros, cons, neutral

## 3. Algorithmes d'explication (Solveurs Gurobi)

### Types d'explications

1. **(1-1)** : Un avantage compense un inconvénient
2. **(1-m)** : Un avantage fort compense plusieurs inconvénients
3. **(m-1)** : Plusieurs avantages compensent un inconvénient
4. **Combinée** : Mix de (1-m) et (m-1)

In [6]:
def solve_explanation_11(omega: np.ndarray, criteria_names: List[str] = None, 
                         eps: float = 1e-6, verbose: bool = False):
    """
    Q1 : Trouve une explication de type (1-1).
    Trade-off (1-1) : une paire (P, C) où un pro compense un con.
    """
    n = len(omega)
    if criteria_names is None:
        criteria_names = [f"C{i}" for i in range(n)]

    pros, cons, neutral = classify_omega(omega, eps=eps)
    
    if sum(omega) <= eps:
        return None, "INVALID"
    if len(cons) == 0:
        return [], "TRIVIAL"
    if len(cons) > len(pros):
        return None, "INFEASIBLE"

    valid_pairs = [(p, c) for p in pros for c in cons if omega[p] + omega[c] > eps]
    
    for c in cons:
        if not any((p, c) in valid_pairs for p in pros):
            return None, "INFEASIBLE"

    m = gp.Model("explain_11")
    m.Params.OutputFlag = 1 if verbose else 0

    z = {(p, c): m.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}") for (p, c) in valid_pairs}
    m.update()

    for c in cons:
        m.addConstr(gp.quicksum(z[p, c] for p in pros if (p, c) in z) == 1)

    for p in pros:
        m.addConstr(gp.quicksum(z[p, c] for c in cons if (p, c) in z) <= 1)

    m.setObjective(gp.quicksum(z[p, c] * (omega[p] + omega[c]) for (p, c) in valid_pairs), GRB.MAXIMIZE)

    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        explanation = [(p, c) for (p, c) in valid_pairs if z[p, c].X > 0.5]
        return explanation, "OPTIMAL"
    if m.status == GRB.INFEASIBLE:
        return None, "INFEASIBLE"
    return None, f"STATUS_{m.status}"


def solve_explanation_1m(omega: np.ndarray, eps: float = 1e-6, verbose: bool = False):
    """
    Q2 : Trouve une explication de type (1-m).
    Trade-off (1-m) : un pro compense plusieurs cons.
    """
    pros, cons, neutral = classify_omega(omega, eps=eps)
    
    if sum(omega) <= eps:
        return None, "INVALID"
    if len(cons) == 0:
        return [], "TRIVIAL"
    if len(pros) == 0:
        return None, "INFEASIBLE"

    m = gp.Model("explain_1m")
    m.Params.OutputFlag = 1 if verbose else 0

    z = {(p, c): m.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}") for p in pros for c in cons}
    y = {p: m.addVar(vtype=GRB.BINARY, name=f"y_{p}") for p in pros}
    m.update()

    M = 10_000

    for c in cons:
        m.addConstr(gp.quicksum(z[p, c] for p in pros) == 1)

    for p in pros:
        for c in cons:
            m.addConstr(z[p, c] <= y[p])

    for p in pros:
        m.addConstr(omega[p] + gp.quicksum(z[p, c] * omega[c] for c in cons) >= eps - M * (1 - y[p]))

    m.setObjective(gp.quicksum(y[p] for p in pros), GRB.MINIMIZE)

    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        explanation = []
        for p in pros:
            if y[p].X > 0.5:
                cons_for_p = [c for c in cons if z[p, c].X > 0.5]
                explanation.append((p, cons_for_p))
        return explanation, "OPTIMAL"
    if m.status == GRB.INFEASIBLE:
        return None, "INFEASIBLE"
    return None, f"STATUS_{m.status}"


def solve_explanation_m1(omega: np.ndarray, eps: float = 1e-6, verbose: bool = False):
    """
    Q3 : Trouve une explication de type (m-1).
    Trade-off (m-1) : plusieurs pros compensent un seul cons.
    """
    pros, cons, neutral = classify_omega(omega, eps=eps)
    
    if sum(omega) <= eps:
        return None, "INVALID"
    if len(cons) == 0:
        return [], "TRIVIAL"
    if len(pros) == 0:
        return None, "INFEASIBLE"

    m = gp.Model("explain_m1")
    m.Params.OutputFlag = 1 if verbose else 0

    z = {(p, c): m.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}") for p in pros for c in cons}
    m.update()

    for p in pros:
        m.addConstr(gp.quicksum(z[p, c] for c in cons) <= 1)

    for c in cons:
        m.addConstr(gp.quicksum(z[p, c] * omega[p] for p in pros) + omega[c] >= eps)

    m.setObjective(gp.quicksum(z[p, c] for p in pros for c in cons), GRB.MINIMIZE)

    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        explanation = []
        for c in cons:
            pros_for_c = [p for p in pros if z[p, c].X > 0.5]
            explanation.append((pros_for_c, c))
        return explanation, "OPTIMAL"
    if m.status == GRB.INFEASIBLE:
        return None, "INFEASIBLE"
    return None, f"STATUS_{m.status}"


def solve_explanation_combined_q4(omega: np.ndarray, criteria_names: List[str] = None, 
                                   eps: float = 1e-6, verbose: bool = False):
    """
    Q4 (combinée) : explication mélangeant trade-offs (1-m) et (m-1).
    """
    n = len(omega)
    if criteria_names is None:
        criteria_names = [f"C{i}" for i in range(n)]

    pros = [i for i, v in enumerate(omega) if v > eps]
    cons = [i for i, v in enumerate(omega) if v < -eps]

    if float(sum(omega)) <= eps:
        return None, None, "INVALID"
    if len(cons) == 0:
        return [], [], "TRIVIAL"
    if len(pros) == 0:
        return None, None, "INFEASIBLE"

    model = gp.Model("explanation_combined_q4")
    model.Params.OutputFlag = 1 if verbose else 0

    M = 10_000.0

    z = {(p, c): model.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}") for p in pros for c in cons}
    mode1m = {c: model.addVar(vtype=GRB.BINARY, name=f"mode1m_{c}") for c in cons}
    y = {p: model.addVar(vtype=GRB.BINARY, name=f"y_{p}") for p in pros}
    w = {(p, c): model.addVar(vtype=GRB.BINARY, name=f"w_{p}_{c}") for p in pros for c in cons}
    has1m = {p: model.addVar(vtype=GRB.BINARY, name=f"has1m_{p}") for p in pros}

    model.update()

    for p in pros:
        for c in cons:
            model.addConstr(z[p, c] <= y[p])

    for c in cons:
        model.addConstr(gp.quicksum(z[p, c] for p in pros) >= 1)

    for c in cons:
        nprosc = gp.quicksum(z[p, c] for p in pros)
        model.addConstr(nprosc <= 1 + M * (1 - mode1m[c]))
        model.addConstr(nprosc >= 2 - M * mode1m[c])

    for c in cons:
        model.addConstr(gp.quicksum(z[p, c] * float(omega[p]) for p in pros) + float(omega[c]) >= eps)

    for p in pros:
        for c in cons:
            model.addConstr(w[p, c] <= z[p, c])
            model.addConstr(w[p, c] <= mode1m[c])
            model.addConstr(w[p, c] >= z[p, c] + mode1m[c] - 1)

    for p in pros:
        sumw = gp.quicksum(w[p, c] for c in cons)
        model.addConstr(has1m[p] <= sumw)
        model.addConstr(sumw <= M * has1m[p])

    for p in pros:
        model.addConstr(float(omega[p]) + gp.quicksum(w[p, c] * float(omega[c]) for c in cons)
                        >= eps - M * (1 - has1m[p]))

    for p in pros:
        for c in cons:
            model.addConstr(gp.quicksum(z[p, c2] for c2 in cons)
                            <= 1 + M * (1 - z[p, c]) + M * mode1m[c])

    model.setObjective(gp.quicksum(z[p, c] for p in pros for c in cons), GRB.MINIMIZE)

    model.optimize()

    if model.status == GRB.INFEASIBLE:
        return None, None, "INFEASIBLE"
    if model.status != GRB.OPTIMAL:
        return None, None, f"STATUS_{model.status}"

    cons_in_1m = [c for c in cons if mode1m[c].X > 0.5]
    cons_in_m1 = [c for c in cons if mode1m[c].X <= 0.5]

    proto_cons = {}
    for c in cons_in_1m:
        for p in pros:
            if z[p, c].X > 0.5:
                proto_cons.setdefault(p, []).append(c)

    tradeoffs_1m = [(p, proto_cons[p]) for p in proto_cons]

    tradeoffs_m1 = []
    for c in cons_in_m1:
        pros_for_c = [p for p in pros if z[p, c].X > 0.5]
        tradeoffs_m1.append((pros_for_c, c))

    return tradeoffs_1m, tradeoffs_m1, "OPTIMAL"

## 4. Fonctions de formatage des explications

In [7]:
def fmt_explanation_11(exp11: List[Tuple[int, int]], criteria_names: List[str], 
                       omega: np.ndarray) -> str:
    """Formate une explication (1-1)."""
    if not exp11:
        return "Dominance totale (aucun inconvénient)."
    
    lines = []
    for p, c in exp11:
        margin = omega[p] + omega[c]
        lines.append(f"  - L'avantage sur '{criteria_names[p]}' (ω={omega[p]:+.2f}) "
                     f"compense l'inconvénient sur '{criteria_names[c]}' (ω={omega[c]:+.2f}) "
                     f"[marge: {margin:+.2f}]")
    return "\n".join(lines)


def fmt_explanation_1m(exp1m: List[Tuple[int, List[int]]], criteria_names: List[str], 
                       omega: np.ndarray) -> str:
    """Formate une explication (1-m)."""
    if not exp1m:
        return "Dominance totale (aucun inconvénient)."
    
    lines = []
    for p, cons_list in exp1m:
        cons_names = ", ".join(f"'{criteria_names[c]}'" for c in cons_list)
        cons_sum = sum(omega[c] for c in cons_list)
        margin = omega[p] + cons_sum
        lines.append(f"  - L'avantage sur '{criteria_names[p]}' (ω={omega[p]:+.2f}) "
                     f"compense les inconvénients sur {cons_names} (Σω={cons_sum:+.2f}) "
                     f"[marge: {margin:+.2f}]")
    return "\n".join(lines)


def fmt_explanation_m1(expm1: List[Tuple[List[int], int]], criteria_names: List[str], 
                       omega: np.ndarray) -> str:
    """Formate une explication (m-1)."""
    if not expm1:
        return "Dominance totale (aucun inconvénient)."
    
    lines = []
    for pros_list, c in expm1:
        pros_names = ", ".join(f"'{criteria_names[p]}'" for p in pros_list)
        pros_sum = sum(omega[p] for p in pros_list)
        margin = pros_sum + omega[c]
        lines.append(f"  - Les avantages sur {pros_names} (Σω={pros_sum:+.2f}) "
                     f"compensent l'inconvénient sur '{criteria_names[c]}' (ω={omega[c]:+.2f}) "
                     f"[marge: {margin:+.2f}]")
    return "\n".join(lines)


def fmt_explanation_combined(trade1m, tradem1, criteria_names: List[str], 
                              omega: np.ndarray) -> str:
    """Formate une explication combinée."""
    lines = []
    
    if trade1m:
        lines.append("Trade-offs (1-m) :")
        for p, cons_list in trade1m:
            cons_names = ", ".join(f"'{criteria_names[c]}'" for c in cons_list)
            cons_sum = sum(omega[c] for c in cons_list)
            margin = omega[p] + cons_sum
            lines.append(f"    - '{criteria_names[p]}' (ω={omega[p]:+.2f}) compense {cons_names} "
                         f"(Σω={cons_sum:+.2f}) [marge: {margin:+.2f}]")
    
    if tradem1:
        lines.append("Trade-offs (m-1) :")
        for pros_list, c in tradem1:
            pros_names = ", ".join(f"'{criteria_names[p]}'" for p in pros_list)
            pros_sum = sum(omega[p] for p in pros_list)
            margin = pros_sum + omega[c]
            lines.append(f"    - {pros_names} (Σω={pros_sum:+.2f}) compensent '{criteria_names[c]}' "
                         f"(ω={omega[c]:+.2f}) [marge: {margin:+.2f}]")
    
    return "\n".join(lines) if lines else "Dominance totale (aucun inconvénient)."

## 5. Fonction principale d'analyse et d'explication

In [8]:
def explain_comparison(name_A: str, name_B: str, solutions: dict, 
                       weights: np.ndarray, criteria_names: List[str]):
    """
    Analyse complète : affiche la comparaison et trouve la meilleure explication.
    """
    A = solutions[name_A]
    B = solutions[name_B]
    
    # Afficher la comparaison
    omega, pros, cons, neutral = display_comparison(name_A, name_B, solutions, weights, criteria_names)
    
    score_A = np.dot(weights, A)
    score_B = np.dot(weights, B)
    
    if score_A <= score_B:
        print(f"\n⚠️  {name_A} n'est PAS meilleur que {name_B} !")
        return None
    
    print(f"\n{'─'*80}")
    print("RECHERCHE D'EXPLICATION")
    print(f"{'─'*80}")
    
    # Essayer Q1 (1-1)
    exp11, st11 = solve_explanation_11(omega, criteria_names=criteria_names)
    if st11 in ["OPTIMAL", "TRIVIAL"]:
        print(f"\n✓ Explication (1-1) trouvée ! [{st11}]")
        print(f"  Longueur : {len(exp11) if exp11 else 0} trade-off(s)\n")
        print(fmt_explanation_11(exp11, criteria_names, omega))
        return {"type": "1-1", "explanation": exp11, "status": st11}
    else:
        print(f"  (1-1) : {st11}")
    
    # Essayer Q2 (1-m)
    exp1m, st1m = solve_explanation_1m(omega)
    if st1m in ["OPTIMAL", "TRIVIAL"]:
        print(f"\n✓ Explication (1-m) trouvée ! [{st1m}]")
        print(f"  Longueur : {len(exp1m) if exp1m else 0} trade-off(s)\n")
        print(fmt_explanation_1m(exp1m, criteria_names, omega))
        return {"type": "1-m", "explanation": exp1m, "status": st1m}
    else:
        print(f"  (1-m) : {st1m}")
    
    # Essayer Q3 (m-1)
    expm1, stm1 = solve_explanation_m1(omega)
    if stm1 in ["OPTIMAL", "TRIVIAL"]:
        print(f"\n✓ Explication (m-1) trouvée ! [{stm1}]")
        print(f"  Longueur : {len(expm1) if expm1 else 0} trade-off(s)\n")
        print(fmt_explanation_m1(expm1, criteria_names, omega))
        return {"type": "m-1", "explanation": expm1, "status": stm1}
    else:
        print(f"  (m-1) : {stm1}")
    
    # Essayer Q4 (combinée)
    trade1m, tradem1, st4 = solve_explanation_combined_q4(omega, criteria_names=criteria_names)
    if st4 in ["OPTIMAL", "TRIVIAL"]:
        n_tradeoffs = len(trade1m or []) + len(tradem1 or [])
        print(f"\n✓ Explication combinée (Q4) trouvée ! [{st4}]")
        print(f"  Longueur : {n_tradeoffs} trade-off(s)\n")
        print(fmt_explanation_combined(trade1m, tradem1, criteria_names, omega))
        return {"type": "combined", "trade1m": trade1m, "tradem1": tradem1, "status": st4}
    else:
        print(f"  Combinée : {st4}")
    
    print("\n✗ Aucune explication trouvée !")
    return None

## 6. Expliquer pourquoi Solution 1 > Solution 2

In [9]:
result_1_vs_2 = explain_comparison('Solution 1', 'Solution 2', solutions, weights, criteria_names)


COMPARAISON : Solution 1 vs Solution 2

Scores : Solution 1 = 19725.30, Solution 2 = 17653.30
Différence : 2072.00

Critère     Poids   Solution 1   Solution 2        Omega Type        
----------------------------------------------------------------------
a             2.0       532.00        94.00      +876.00 PRO         
b             1.0       120.00      -756.00      +876.00 PRO         
c             5.0       330.00       174.40      +778.00 PRO         
d             3.0        54.00      -254.67      +926.00 PRO         
e             4.0        12.00       225.50      -854.00 CON         
f             6.0       125.00       162.33      -224.00 CON         
g             1.0       320.00       670.00      -350.00 CON         
h             7.0        59.40        19.97      +276.00 PRO         
i             4.0       125.00       127.50       -10.00 CON         
j             3.0       325.00        37.67      +862.00 PRO         
k             8.0        77.00       139.0

## 7. Expliquer pourquoi Solution 1 > Solution 3

In [10]:
result_1_vs_3 = explain_comparison('Solution 1', 'Solution 3', solutions, weights, criteria_names)


COMPARAISON : Solution 1 vs Solution 3

Scores : Solution 1 = 19725.30, Solution 3 = -15731.30
Différence : 35456.60

Critère     Poids   Solution 1   Solution 3        Omega Type        
----------------------------------------------------------------------
a             2.0       532.00       174.00      +716.00 PRO         
b             1.0       120.00       652.00      -532.00 CON         
c             5.0       330.00     -1094.00     +7120.00 PRO         
d             3.0        54.00      -176.00      +690.00 PRO         
e             4.0        12.00      -854.00     +3464.00 PRO         
f             6.0       125.00      -573.00     +4188.00 PRO         
g             1.0       320.00         6.00      +314.00 PRO         
h             7.0        59.40        50.60       +61.60 PRO         
i             4.0       125.00       515.00     -1560.00 CON         
j             3.0       325.00      -395.00     +2160.00 PRO         
k             8.0        77.00       341

## 8. Expliquer pourquoi Solution 1 > Solution 4

In [11]:
result_1_vs_4 = explain_comparison('Solution 1', 'Solution 4', solutions, weights, criteria_names)


COMPARAISON : Solution 1 vs Solution 4

Scores : Solution 1 = 19725.30, Solution 4 = 15711.30
Différence : 4014.00

Critère     Poids   Solution 1   Solution 4        Omega Type        
----------------------------------------------------------------------
a             2.0       532.00       604.00      -144.00 CON         
b             1.0       120.00       976.00      -856.00 CON         
c             5.0       330.00       680.00     -1750.00 CON         
d             3.0        54.00       -52.00      +318.00 PRO         
e             4.0        12.00      -234.00      +984.00 PRO         
f             6.0       125.00       333.00     -1248.00 CON         
g             1.0       320.00      -254.00      +574.00 PRO         
h             7.0        59.40      -608.60     +4676.00 PRO         
i             4.0       125.00      -523.00     +2592.00 PRO         
j             3.0       325.00       277.00      +144.00 PRO         
k             8.0        77.00      -493.0

## 9. Résumé des explications

In [12]:
print("="*80)
print("RÉSUMÉ : Types d'explications pour chaque comparaison")
print("="*80)

comparisons = [
    ('Solution 1', 'Solution 2'),
    ('Solution 1', 'Solution 3'),
    ('Solution 1', 'Solution 4'),
]

results = []

for name_A, name_B in comparisons:
    A = solutions[name_A]
    B = solutions[name_B]
    omega = compute_omega(A, B, weights)
    pros, cons, _ = classify_omega(omega)
    
    # Tester les différents types
    _, st11 = solve_explanation_11(omega, criteria_names=criteria_names)
    _, st1m = solve_explanation_1m(omega)
    _, stm1 = solve_explanation_m1(omega)
    _, _, st4 = solve_explanation_combined_q4(omega, criteria_names=criteria_names)
    
    # Dominance ?
    dom = 'OK' if len(cons) == 0 else '-'
    
    results.append({
        'Comparaison': f"{name_A} > {name_B}",
        '#Pros': len(pros),
        '#Cons': len(cons),
        'Dom': dom,
        '(1-1)': 'OK' if st11 == 'OPTIMAL' else '-',
        '(1-m)': 'OK' if st1m == 'OPTIMAL' else '-',
        '(m-1)': 'OK' if stm1 == 'OPTIMAL' else '-',
        'Comb': 'OK' if st4 == 'OPTIMAL' else '-'
    })

df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))

RÉSUMÉ : Types d'explications pour chaque comparaison
            Comparaison  #Pros  #Cons Dom (1-1) (1-m) (m-1) Comb
Solution 1 > Solution 2     13     14   -     -    OK     -   OK
Solution 1 > Solution 3     20      7   -    OK    OK    OK   OK
Solution 1 > Solution 4     14     13   -     -     -     -   OK


## 10. Statistiques d'explicabilité sur toutes les paires

In [13]:
def calculer_statistiques_toutes_paires():
    """
    Calcule les statistiques d'explicabilité pour toutes les paires de solutions.
    """
    paires = []
    for i, name_A in enumerate(solution_names):
        for j, name_B in enumerate(solution_names):
            if i != j:
                score_A = scores[name_A]
                score_B = scores[name_B]
                if score_A > score_B:
                    paires.append((name_A, name_B))
    
    stats = {
        "total": len(paires),
        "dominance": 0,
        "q1_11": 0,
        "q2_1m": 0,
        "q3_m1": 0,
        "q4_combined": 0,
        "non_explicable": 0
    }
    
    for name_A, name_B in paires:
        A = solutions[name_A]
        B = solutions[name_B]
        omega = compute_omega(A, B, weights)
        _, cons, _ = classify_omega(omega)
        
        if len(cons) == 0:
            stats["dominance"] += 1
            continue
        
        _, st11 = solve_explanation_11(omega, criteria_names=criteria_names)
        if st11 == "OPTIMAL":
            stats["q1_11"] += 1
            continue
        
        _, st1m = solve_explanation_1m(omega)
        if st1m == "OPTIMAL":
            stats["q2_1m"] += 1
            continue
        
        _, stm1 = solve_explanation_m1(omega)
        if stm1 == "OPTIMAL":
            stats["q3_m1"] += 1
            continue
        
        _, _, st4 = solve_explanation_combined_q4(omega, criteria_names=criteria_names)
        if st4 == "OPTIMAL":
            stats["q4_combined"] += 1
            continue
        
        stats["non_explicable"] += 1
    
    return stats


print("Calcul des statistiques d'explicabilité...")
stats = calculer_statistiques_toutes_paires()

explicable = stats["total"] - stats["non_explicable"]

print(f"\n{'='*65}")
print("STATISTIQUES D'EXPLICABILITÉ - DATA 27 CRITÈRES")
print(f"{'='*65}")
print(f"\nSur {stats['total']} comparaisons analysées :")
print("\n{:<35} {:>10} {:>10}".format("Type d'explication", "Nombre", "%"))
print("-" * 57)
print(f"{'Dominance (aucun compromis)':<35} {stats['dominance']:>10} {100*stats['dominance']/max(1,stats['total']):>9.1f}%")
print(f"{'Q1 : 1-1 (échange simple)':<35} {stats['q1_11']:>10} {100*stats['q1_11']/max(1,stats['total']):>9.1f}%")
print(f"{'Q2 : 1-m (argument fort)':<35} {stats['q2_1m']:>10} {100*stats['q2_1m']/max(1,stats['total']):>9.1f}%")
print(f"{'Q3 : m-1 (accumulation)':<35} {stats['q3_m1']:>10} {100*stats['q3_m1']/max(1,stats['total']):>9.1f}%")
print(f"{'Q4 : combinée':<35} {stats['q4_combined']:>10} {100*stats['q4_combined']/max(1,stats['total']):>9.1f}%")
print("-" * 57)
print(f"{'TOTAL EXPLICABLE':<35} {explicable:>10} {100*explicable/max(1,stats['total']):>9.1f}%")
print(f"{'Non explicable':<35} {stats['non_explicable']:>10} {100*stats['non_explicable']/max(1,stats['total']):>9.1f}%")

Calcul des statistiques d'explicabilité...

STATISTIQUES D'EXPLICABILITÉ - DATA 27 CRITÈRES

Sur 6 comparaisons analysées :

Type d'explication                      Nombre          %
---------------------------------------------------------
Dominance (aucun compromis)                  0       0.0%
Q1 : 1-1 (échange simple)                    3      50.0%
Q2 : 1-m (argument fort)                     1      16.7%
Q3 : m-1 (accumulation)                      0       0.0%
Q4 : combinée                                2      33.3%
---------------------------------------------------------
TOTAL EXPLICABLE                             6     100.0%
Non explicable                               0       0.0%
